---

### 🎓 **Professor**: Apostolos Filippas

### 📘 **Class**: AI Engineering

### 📋 **Homework 3**: Improving Lexical Search

### 📅 **Due Date**: Day of Lecture 4, 11:59 PM


**Note**: You are not allowed to share the contents of this notebook with anyone outside this class without written permission by the professor.

---

You'll apply what we covered in Lecture 3 (Lexical Search & BM25) to a real e-commerce search problem using the **WANDS dataset** 
- WANDS stands for Wayfair Annotated Dataset. It's a dataset of furniture products and search queries, along with human relevance judgments.

You will:
1. **Build a search engine** from scratch using BM25.
2. **Learn how to evaluate search results** using NDCG — a metric for measuring search quality
3. **Attempt to improve your search engine** by adding multiple fields
4. **Use LLMs to improve your search engine** by adding simple query understanding

Yes, *you* will do all these things. Let's go!

---

## Task 1: Environment Setup

First, let's set up your environment and verify everything works.

### 1a. Install dependencies and verify imports

Run `uv add pystemmer` in your terminal to add the Snowball stemmer. Then run the cell below to verify all imports work.

In [2]:
# Task 1a: Verify imports work
import pandas as pd
import numpy as np
from collections import Counter
import string
from pathlib import Path
# a stemmer from `pystemmer` for better tokenization
import Stemmer 
# llm packages
import litellm
from pydantic import BaseModel, Field
from typing import Optional
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Pandas display settings
# pd.set_option('display.max_colwidth', 100)

# Ignore pydantic warnings for litellm
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

print("All imports successful!")

All imports successful!


### 1b. Verify API keys

Test that your API keys work by making a simple call.

In [3]:
# Task 1b: Verify API keys
response = litellm.completion(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say 'API working!' and nothing else."}],
    max_tokens=20
)
print(response.choices[0].message.content)

API working!


---

## Task 2: Load and Explore the WANDS Dataset

The **WANDS dataset** (Wayfair Annotated Dataset) contains:
- 43K furniture products from Wayfair
- 480 real search queries
- 233K human relevance judgments (query-product pairs)

This is a real-world search benchmark used to evaluate e-commerce search systems!

**Data Source**: [WANDS on GitHub](https://github.com/wayfair/WANDS)

The data files are pre-downloaded in the `data/` directory:
- `wayfair-products.csv` - Product catalog
- `wayfair-queries.csv` - Search queries
- `wayfair-labels.csv` - Relevance judgments

### Data Loading Functions (provided)

Run the cell below to define the loading functions.

In [21]:
# Data loading functions (provided)
# Note: Data from WANDS (Wayfair Annotated Dataset)
# Source: https://github.com/wayfair/WANDS

def load_wands_products(data_dir: str = "../data") -> pd.DataFrame:
    """
    Load WANDS products from local file.
    
    Args:
        data_dir: Path to the data directory containing wayfair-products.csv
        
    Returns:
        DataFrame with product information including product_id, product_name,
        product_class, category_hierarchy, product_description, etc.
    """
    filepath = Path(data_dir) / "wayfair-products.csv"
    products = pd.read_csv(filepath, sep='\t')
    products = products.rename(columns={'category hierarchy': 'category_hierarchy'})
    return products

def load_wands_queries(data_dir: str = "../data") -> pd.DataFrame:
    """
    Load WANDS queries from local file.
    
    Args:
        data_dir: Path to the data directory containing wayfair-queries.csv
        
    Returns:
        DataFrame with query_id and query columns
    """
    filepath = Path(data_dir) / "wayfair-queries.csv"
    queries = pd.read_csv(filepath, sep='\t')
    return queries

def load_wands_labels(data_dir: str = "../data") -> pd.DataFrame:
    """
    Load WANDS relevance labels from local file.
    
    Args:
        data_dir: Path to the data directory containing wayfair-labels.csv
        
    Returns:
        DataFrame with query_id, product_id, label (Exact/Partial/Irrelevant),
        and grade (2/1/0) columns
    """
    filepath = Path(data_dir) / "wayfair-labels.csv"
    labels = pd.read_csv(filepath, sep='\t')
    grade_map = {'Exact': 2, 'Partial': 1, 'Irrelevant': 0}
    labels['grade'] = labels['label'].map(grade_map)
    return labels

print("Loading functions defined!")

Loading functions defined!


### 2a. Load the data

Use the provided functions to load all three datasets. Print the number of rows in each.

In [22]:
# Task 2a: Load the data

# YOUR CODE HERE
stemmer = Stemmer.Stemmer("english")

def stemming_tokenize(text: str):
    # 1. lowercase
    text = text.lower()
    
    # 2. remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # 3. split into tokens
    tokens = text.split()
    
    # 4. stem tokens
    tokens = stemmer.stemWords(tokens)
    
    return tokens



In [23]:
# Test example 1
stemming_tokenize("Turquoise Pillows!")


['turquois', 'pillow']

In [16]:
stemming_tokenize("Apostolos complaints")


['apostolo', 'complaint']

### 2b. Explore products

List the available columns, and display a few sample products. 

Which columns might be useful for search?

In [27]:
products = load_wands_products()


In [28]:
# 2b: Explore products

print("Products shape:", products.shape)
print("\nColumns:")
print(list(products.columns))

products.head(5)


Products shape: (42994, 9)

Columns:
['product_id', 'product_name', 'product_class', 'category_hierarchy', 'product_description', 'product_features', 'rating_count', 'average_rating', 'review_count']


,product_id,product_name,product_class,category_hierarchy,product_description,product_features,rating_count,average_rating,review_count
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0


In [29]:
# Look at the key text fields we might search over
cols_to_view = [
    "product_id",
    "product_name",
    "product_class",
    "category_hierarchy",
    "product_description",
    "product_features",
]

# show a few rows (some descriptions/features can be long)
products[cols_to_view].sample(5, random_state=42)


,product_id,product_name,product_class,category_hierarchy,product_description,product_features
32193,32193,woven wire basket,"Boxes, Bins, Baskets, & Buckets",Storage & Organization / Storage Containers & ...,NaN,overalldepth-fronttoback:18|producttype : bask...
21379,21379,scallop nightstand,Kids Nightstands,Baby & Kids / Toddler & Kids Bedroom Furniture...,the scallop collection shows off its charming ...,framematerialdetails : mdf and plywood|theme :...
42275,42275,ybm home stainless steel colander,"Strainers, Colanders, & Salad Spinners",Kitchen & Tabletop / Kitchen Utensils & Tools ...,this stainless steel colander with side handle...,overalldepth-fronttoback:11|overallproductweig...
29885,29885,oneida 2 piece stainless steel strainer set,"Strainers, Colanders, & Salad Spinners",Kitchen & Tabletop / Kitchen Utensils & Tools ...,NaN,producttype : strainer set|materialfinish : st...
8194,8194,ryland platform bed frame,Bed Frames|Beds,NaN,update the look of your room by adding the uph...,color : black|overallwidth-sidetoside:60|overa...


In [30]:
# Task 2b: Explore products
# List available columns
print("Product columns:", list(products.columns))

# Display a few sample products
products.head()


Product columns: ['product_id', 'product_name', 'product_class', 'category_hierarchy', 'product_description', 'product_features', 'rating_count', 'average_rating', 'review_count']


,product_id,product_name,product_class,category_hierarchy,product_description,product_features,rating_count,average_rating,review_count
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0


In [31]:
# Task 2b (continued): Which columns are useful for search?
# Show sample of text columns we can index with BM25:
display_cols = ["product_name", "product_class", "category_hierarchy", "product_description"]
products[display_cols].head(3)

# Useful for search: product_name, product_description, category_hierarchy, product_features
# (all contain query-relevant text; we'll concatenate them when building the index)


,product_name,product_class,category_hierarchy,product_description
0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha..."
1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend..."
2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...


In [8]:
# YOUR CODE HERE


### 2c. Understand relevance judgments

The `labels` dataset contains human judgments of relevance. In particular, for each query-product pair, it contains:
| Label        | Grade | Description                                 |
|--------------|-------|---------------------------------------------|
| Exact        |   2   | This product is exactly what the user wants |
| Partial      |   1   | This product is somewhat relevant           |
| Irrelevant   |   0   | This product doesn't match the query        |

First, let's look at the distribution of grades.

In [32]:
# Task 2c: Understand judgments

# YOUR CODE HERE
labels = load_wands_labels()
labels.head()



,id,query_id,product_id,label,grade
0,0,0,25434,Exact,2
1,1,0,12088,Irrelevant,0
2,2,0,42931,Exact,2
3,3,0,2636,Exact,2
4,4,0,42923,Exact,2


In [33]:
# YOUR CODE HERE
# Task 2c: Understand judgments

# Basic check
print("Labels shape:", labels.shape)
print("Columns:", list(labels.columns))

# Distribution of labels and grades
label_counts = labels["label"].value_counts()
grade_counts = labels["grade"].value_counts().sort_index()

print("\nLabel counts:")
print(label_counts)

print("\nGrade counts (0=Irrelevant, 1=Partial, 2=Exact):")
print(grade_counts)

print("\nGrade percentages:")
print((grade_counts / grade_counts.sum() * 100).round(2))


Labels shape: (233448, 5)
Columns: ['id', 'query_id', 'product_id', 'label', 'grade']

Label counts:
label
Partial       146633
Irrelevant     61201
Exact          25614
Name: count, dtype: int64

Grade counts (0=Irrelevant, 1=Partial, 2=Exact):
grade
0     61201
1    146633
2     25614
Name: count, dtype: int64

Grade percentages:
grade
0    26.22
1    62.81
2    10.97
Name: count, dtype: float64


In [34]:
# Sanity: how many queries/products are covered by labels?
print("Unique queries in labels:", labels["query_id"].nunique())
print("Unique products in labels:", labels["product_id"].nunique())

# Show a few examples from each grade
for g in [2, 1, 0]:
    sample = labels[labels["grade"] == g].sample(5, random_state=42)
    print(f"\nExamples with grade={g}:")
    display(sample)


Unique queries in labels: 480
Unique products in labels: 42986

Examples with grade=2:


,id,query_id,product_id,label,grade
42160,42266,414,3190,Exact,2
28826,28905,228,11756,Exact,2
24884,24888,167,36510,Exact,2
16259,16259,121,40519,Exact,2
43613,43719,420,15103,Exact,2



Examples with grade=1:


,id,query_id,product_id,label,grade
36714,36793,327,23218,Partial,1
188013,188580,441,6976,Partial,1
11644,11644,90,35576,Partial,1
15500,15500,116,8736,Partial,1
74227,74378,78,11385,Partial,1



Examples with grade=0:


,id,query_id,product_id,label,grade
208491,209058,14,16648,Irrelevant,0
215207,215774,40,2101,Irrelevant,0
212573,213140,26,27857,Irrelevant,0
106972,107124,211,7046,Irrelevant,0
227691,228258,124,16664,Irrelevant,0


In [35]:
labels.groupby(["label", "grade"]).size()


label       grade
Exact       2         25614
Irrelevant  0         61201
Partial     1        146633
dtype: int64

---
## Task 3: Build and Run BM25 Search

Now let's build a BM25 search engine! We'll use the same concepts from Lecture 3.

### Provided Functions

We're giving you these functions to work with. Run the next cell to define them, then look at the examples.

| Function | What it does |
|----------|--------------|
| `snowball_tokenize(text)` | Tokenizes text, removes punctuation, stems words |
| `build_index(docs, tokenizer)` | Builds an inverted index from a list of documents |
| `get_tf(term, doc_id, index)` | Gets term frequency for a term in a document |
| `get_df(term, index)` | Gets document frequency for a term (how many docs contain the term) |
| `bm25_idf(df, num_docs)` | Calculates the IDF component of BM25 |
| `bm25_tf(tf, doc_len, avg_doc_len)` | Calculates the TF normalization for BM25 |
| `score_bm25(query, index, ...)` | Scores all documents for a query using BM25 |
| `search_products(query, ...)` | Searches and returns top-k results |

In [36]:
# Provided functions - run this cell to define them

stemmer = Stemmer.Stemmer('english')
punct_trans = str.maketrans({key: ' ' for key in string.punctuation})

def snowball_tokenize(text: str) -> list[str]:
    """
    Tokenize text with Snowball stemming.
    
    Args:
        text: The text to tokenize
        
    Returns:
        List of stemmed tokens
    """
    if pd.isna(text) or text is None:
        return []
    text = str(text).translate(punct_trans)
    tokens = text.lower().split()
    return [stemmer.stemWord(token) for token in tokens]

def build_index(docs: list[str], tokenizer) -> tuple[dict, list[int]]:
    """
    Build an inverted index from a list of documents.
    
    Args:
        docs: List of document strings to index
        tokenizer: Function that takes text and returns list of tokens
        
    Returns:
        index: dict mapping term -> {doc_id: term_count}
        doc_lengths: list of document lengths (in tokens)
    """
    index = {}
    doc_lengths = []
    
    for doc_id, doc in enumerate(docs):
        tokens = tokenizer(doc)
        doc_lengths.append(len(tokens))
        term_counts = Counter(tokens)
        
        for term, count in term_counts.items():
            if term not in index:
                index[term] = {}
            index[term][doc_id] = count
    
    return index, doc_lengths

def get_tf(term: str, doc_id: int, index: dict) -> int:
    """
    Get term frequency for a term in a document.
    
    Args:
        term: The term to look up
        doc_id: The document ID
        index: The inverted index
        
    Returns:
        Term frequency (count), or 0 if not found
    """
    if term in index and doc_id in index[term]:
        return index[term][doc_id]
    return 0

def get_df(term: str, index: dict) -> int:
    """
    Get document frequency for a term.
    
    Args:
        term: The term to look up
        index: The inverted index
        
    Returns:
        Number of documents containing the term
    """
    if term in index:
        return len(index[term])
    return 0

def bm25_idf(df: int, num_docs: int) -> float:
    """
    BM25 IDF formula.
    
    Args:
        df: Document frequency
        num_docs: Total number of documents
        
    Returns:
        IDF score
    """
    return np.log((num_docs - df + 0.5) / (df + 0.5) + 1)

def bm25_tf(tf: int, doc_len: int, avg_doc_len: float, k1: float = 1.2, b: float = 0.75) -> float:
    """
    BM25 TF normalization.
    
    Args:
        tf: Term frequency
        doc_len: Document length in tokens
        avg_doc_len: Average document length
        k1: Saturation parameter (default 1.2)
        b: Length normalization (default 0.75)
        
    Returns:
        Normalized TF score
    """
    return (tf * (k1 + 1)) / (tf + k1 * (1 - b + b * doc_len / avg_doc_len))

def score_bm25(query: str, index: dict, num_docs: int, doc_lengths: list[int], 
               tokenizer, k1: float = 1.2, b: float = 0.75) -> np.ndarray:
    """
    Score all documents using BM25.
    
    Args:
        query: The search query
        index: Inverted index
        num_docs: Total number of documents
        doc_lengths: List of document lengths
        tokenizer: Tokenization function
        
    Returns:
        Array of scores for each document
    """
    query_tokens = tokenizer(query)
    scores = np.zeros(num_docs)
    avg_doc_len = np.mean(doc_lengths) if doc_lengths else 1.0
    
    for token in query_tokens:
        df = get_df(token, index)
        if df == 0:
            continue
        
        idf = bm25_idf(df, num_docs)
        
        if token in index:
            for doc_id, tf in index[token].items():
                tf_norm = bm25_tf(tf, doc_lengths[doc_id], avg_doc_len, k1, b)
                scores[doc_id] += idf * tf_norm
    
    return scores

def search_products(query: str, products_df: pd.DataFrame, index: dict, 
                    doc_lengths: list[int], tokenizer, k: int = 10) -> pd.DataFrame:
    """
    Search products and return top-k results.
    
    Args:
        query: The search query
        products_df: DataFrame of products
        index: Inverted index
        doc_lengths: Document lengths
        tokenizer: Tokenization function
        k: Number of results to return
        
    Returns:
        DataFrame with top-k products and scores
    """
    scores = score_bm25(query, index, len(products_df), doc_lengths, tokenizer)
    top_k_idx = np.argsort(-scores)[:k]
    
    results = products_df.iloc[top_k_idx].copy()
    results['score'] = scores[top_k_idx]
    results['rank'] = range(1, k + 1)
    return results

print("All functions defined!")

All functions defined!


In [37]:
# Examples of each function

# 1. snowball_tokenize - tokenizes and stems text
print("1. snowball_tokenize('Running shoes are amazing!')")
print(f"   -> {snowball_tokenize('Running shoes are amazing!')}")
print("   Notice: 'Running' -> 'run', 'shoes' -> 'shoe', 'amazing' -> 'amaz'")

# 2. build_index - builds inverted index (we'll use a tiny example)
tiny_docs = ["red shoe", "blue shoe", "red hat"]
tiny_index, tiny_lengths = build_index(tiny_docs, snowball_tokenize)
print("\n2. build_index(['red shoe', 'blue shoe', 'red hat'], tokenizer)")
print(f"   Index: {tiny_index}")
print(f"   Lengths: {tiny_lengths}")

# 3. get_tf - get term frequency
print("\n3. get_tf('red', doc_id=0, tiny_index)")
print(f"   -> {get_tf('red', 0, tiny_index)}  (doc 0 = 'red shoe' has 1 'red')")

# 4. get_df - get document frequency  
print("\n4. get_df('red', tiny_index)")
print(f"   -> {get_df('red', tiny_index)}  ('red' appears in 2 documents)")

# 5. bm25_idf - calculate IDF (rare terms get higher scores)
print("\n5. bm25_idf(df=100, num_docs=10000)")
print(f"   -> {bm25_idf(100, 10000):.4f}  (term in 100 of 10000 docs)")

# 6. bm25_tf - normalize term frequency by document length
print("\n6. bm25_tf(tf=3, doc_len=50, avg_doc_len=100)")
print(f"   -> {bm25_tf(3, 50, 100):.4f}  (short doc gets boosted)")

# 7-8. score_bm25 and search_products - we'll use these next!
print("\nWe'll use score_bm25() and search_products() in Task 3a!")

1. snowball_tokenize('Running shoes are amazing!')
   -> ['run', 'shoe', 'are', 'amaz']
   Notice: 'Running' -> 'run', 'shoes' -> 'shoe', 'amazing' -> 'amaz'

2. build_index(['red shoe', 'blue shoe', 'red hat'], tokenizer)
   Index: {'red': {0: 1, 2: 1}, 'shoe': {0: 1, 1: 1}, 'blue': {1: 1}, 'hat': {2: 1}}
   Lengths: [2, 2, 2]

3. get_tf('red', doc_id=0, tiny_index)
   -> 1  (doc 0 = 'red shoe' has 1 'red')

4. get_df('red', tiny_index)
   -> 2  ('red' appears in 2 documents)

5. bm25_idf(df=100, num_docs=10000)
   -> 4.6003  (term in 100 of 10000 docs)

6. bm25_tf(tf=3, doc_len=50, avg_doc_len=100)
   -> 1.7600  (short doc gets boosted)

We'll use score_bm25() and search_products() in Task 3a!


### 3a. Create BM25 index for product_name

Build an inverted index for the `product_name` field and run a sample search for a product.

In [38]:
# Task 3a: Create BM25 index for product_name

# YOUR CODE HERE
# pick fields that exist in your products dataframe
fields = [c for c in ["product_name","product_class","category_hierarchy","product_description","product_features"]
          if c in products.columns]

# create one searchable text per product
product_docs = (
    products[fields]
    .fillna("")
    .agg(" ".join, axis=1)
    .tolist()
)

print("Num docs:", len(product_docs))
print(product_docs[0][:300])


Num docs: 42994
solid wood platform bed Beds Furniture / Bedroom Furniture / Beds & Headboards / Beds / Twin Beds good , deep sleep can be quite difficult to have in this busy age . fortunately , there ’ s an antidote to such a problem : a nice , quality bed frame like the acacia kaylin . solidly constructed from a


### 3b. Add product_description to search

Create a second index for `product_description` and combine scores from both fields.

**Hint**: You can combine the two scores by adding them together. This is like multi-field search from Lecture 3.

In [39]:
# Task 3b: Add product_description to search

# YOUR CODE HERE
index, doc_lens = build_index(product_docs, snowball_tokenize)
print("Index built. Unique terms:", len(index))


Index built. Unique terms: 52047


In [41]:
search_products(
    "turquoise pillows",
    products,
    index,
    doc_lens,                 # ← THIS was missing
    tokenizer=snowball_tokenize,
    k=10
)


,product_id,product_name,product_class,category_hierarchy,product_description,product_features,rating_count,average_rating,review_count,score,rank
5998,5998,noriega turquoise geometric pillow,Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,make a bold statement in your lounge area or b...,covermaterial : polyester/polyester blend|reve...,2.0,4.0,1.0,11.948099,1
17920,17920,diana knit zig-zag textured turquoise throw,Blankets And Throws,Décor & Pillows / Decorative Pillows & Blanket...,this diana knit zig-zag textured turquoise thr...,lifestage : adult|color : turquoise|materialde...,361.0,5.0,237.0,11.329549,2
4676,4676,peacock inspired single iron wall sconce,Candle Holders,Décor & Pillows / Candles & Holders / Candle H...,your room will sparkle with gorgeous candlelig...,overallproductweight:0.6|holdermaterial : iron...,104.0,4.5,88.0,10.850023,3
4673,4673,peacock blossom duo cup iron wall sconce,Candle Holders,Décor & Pillows / Candles & Holders / Candle H...,"twice the candlelight , twice the charm ! this...",overallproductweight:1.2|warrantylength:30 day...,65.0,4.5,49.0,10.808407,4
23599,23599,patel animal print tortoise turquoise indoor /...,Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,upgrade your outdoor living space with these s...,overalldepth-fronttoback:7|overallproductweigh...,15.0,5.0,11.0,10.794032,5
11792,11792,buffalo throw pillow,Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,this colorful square reversible buffalo throw ...,contrastingbordercolor : orange|edgetype : pip...,1.0,5.0,1.0,10.794032,6
4674,4674,peacock blossom single iron wall sconce,Candle Holders,Décor & Pillows / Candles & Holders / Candle H...,shimmering turquoise jewels and a blossom glow...,candlecompatibility : votive/tealight|holderpl...,57.0,5.0,42.0,10.645513,7
4675,4675,peacock inspired trio small iron votive holder,Candle Holders,Décor & Pillows / Candles & Holders / Candle H...,this centerpiece will shimmer and shine with s...,dssecondaryproductstyle : ornate traditional|c...,42.0,4.5,30.0,10.645513,8
21236,21236,"desert medley indoor/outdoor , removable cover...",Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,"this boho-inspired collection combines kiwi , ...",overallheight-toptobottom:6|decorativeaddition...,NaN,NaN,NaN,10.094675,9
42633,42633,peacock iron wall sconce,Candle Holders,Décor & Pillows / Candles & Holders / Candle H...,decorate your walls with color and light with ...,color : teal|style : eclectic|overalldepth-fro...,6.0,5.0,6.0,10.029641,10


In [42]:
len(doc_lens), len(products)


(42994, 42994)

---

## Task 4: Measuring Search Quality

We built a little search engine. How do we know if it's any good?

Consider two search results for "coffee table":

| Ranking A | Ranking B |
|-----------|-----------|
| 1. Wooden Coffee Table (Exact) | 1. Metal Lamp (Irrelevant) |
| 2. Glass Coffee Table (Exact) | 2. Wooden Coffee Table (Exact) |
| 3. Metal Lamp (Irrelevant) | 3. Glass Coffee Table (Exact) |

### A. Precision

One way to measure the quality of a ranking is to look at the precision within these first 3 results. 
- Precision is the ratio of relevant results to total results at position k.
- We call this precision@3, and more generally precision@k is the ratio of relevant results to total results at position k.
  
In this scenario, if we consider "exact" results as relevant, then both rankings have precision@3 = 2/3.

### B. DCG

Both rankings have the same precision, but Ranking A is clearly better 
- users look at results from the top down, and most people never scroll past the first few results
- as such, rankings that return relevant results earlier are better

So we need a metric that rewards **relevant** results, and rewards them **more** when they appear at the **top**

NDCG (Normalized Discounted Cumulative Gain) does this by giving each result a "gain" based on its relevance, then **discounting** that gain based on position.

**The formula** for each result at position $i$:

$$\text{gain}_i = \frac{2^{\text{relevance}} - 1}{\log_2(i + 1)}$$

- **Numerator** $(2^{\text{relevance}} - 1)$: How relevant is this result?
  - Irrelevant (0): $2^0 - 1 = 0$ (no gain)
  - Partial (1): $2^1 - 1 = 1$ (some gain)
  - Exact (2): $2^2 - 1 = 3$ (lots of gain!)
  
- **Denominator** $\log_2(i + 1)$: The "discount" based on position
  - Position 1: $\log_2(2) = 1$ (no discount)
  - Position 2: $\log_2(3) = 1.58$ (small discount)
  - Position 10: $\log_2(11) = 3.46$ (bigger discount)

**DCG** sums the discounted score for each result

$$\text{DCG} = \sum_{i=1}^{k} \frac{2^{\text{relevance}_i} - 1}{\log_2(i + 1)}$$

### 3. NDCG: Normalized DCG

One problem with DCG is that the score depends on how many relevant products exist. 
- A query with 10 exact matches will have a higher DCG than one with only 2, even if both rankings are "perfect."

One solution is to normalize by the *ideal* DCG — what the score would be if we ranked everything perfectly (all relevant results at the top).

$$\text{NDCG} = \frac{\text{DCG}}{\text{Ideal DCG}}$$

- **NDCG = 1.0**: Perfect -- best possible order
- **NDCG = 0.5**: OK -- some good some bad
- **NDCG = 0.0**: Worst -- results are irrelevant

**Read the above carefully.** In the next cell, explain in your own words: why does the discount formula use $\log_2$? What happens to results at position 1 vs position 10?

In [ ]:
# Task 4a: Answer in a comment
# Why does DCG use log2 for the discount? What's the effect on position 1 vs position 10?
#
# YOUR ANSWER HERE
## DCG uses a log2 discount to model how users pay much more attention to top results
# and rapidly less attention as they go down the ranking. The log function gives
# diminishing penalties: moving from rank 1→2 matters a lot; rank 9→10 matters less.
# Using log2 is convenient because it makes the discount at position 1 equal to 1
# (log2(2)=1), so the top result isn't discounted.
# Effect: position 1 gets full weight; position 10 is discounted by log2(11)≈3.46,
# so its contribution is about 1/3.46 ≈ 0.29 of the same relevance at position 1.


### 4b. Calculate NDCG by hand

Let's work through an example step by step.

**Scenario**: You search for "wooden coffee table" and get these results:

| Position | Product | Relevance |
|----------|---------|----------|
| 1 | Glass Coffee Table | Partial (1) |
| 2 | Wooden Coffee Table | Exact (2) |
| 3 | Wooden Side Table | Partial (1) |
| 4 | Metal Coffee Table | Irrelevant (0) |
| 5 | Wooden Coffee Table (different) | Exact (2) |

**Your task**: Calculate DCG and NDCG@5 by hand.

In [ ]:
# Task 4b: Calculate NDCG by hand

# YOUR CODE HERE
# # Task 4b: DCG and NDCG@5 (hand calculation, no code execution)

# Given relevance scores at ranks 1–5:
# [1, 2, 1, 0, 2]

# DCG formula:
# DCG@k = sum_{i=1..k} (2^rel_i - 1) / log2(i + 1)

# DCG@5 calculation:
# Rank 1 (rel=1): (2^1 - 1) / log2(2) = 1 / 1 = 1.000
# Rank 2 (rel=2): (2^2 - 1) / log2(3) = 3 / 1.585 = 1.893
# Rank 3 (rel=1): (2^1 - 1) / log2(4) = 1 / 2 = 0.500
# Rank 4 (rel=0): (2^0 - 1) / log2(5) = 0
# Rank 5 (rel=2): (2^2 - 1) / log2(6) = 3 / 2.585 = 1.161

# DCG@5 = 1.000 + 1.893 + 0.500 + 0 + 1.161 = 4.553

# IDCG@5 calculation:
# Ideal ordering of relevance scores: [2, 2, 1, 1, 0]

# Rank 1: 3 / 1 = 3.000
# Rank 2: 3 / 1.585 = 1.893
# Rank 3: 1 / 2 = 0.500
# Rank 4: 1 / 2.322 = 0.431
# Rank 5: 0 / 2.585 = 0

# IDCG@5 = 5.823

# NDCG@5 = DCG@5 / IDCG@5 = 4.553 / 5.823 ≈ 0.782


### 4c. Implement NDCG function

Now implement the NDCG calculation in code. Verify your implementation matches your hand calculation!

In [47]:
# Task 4c: Implement NDCG function

# YOUR CODE HERE
import numpy as np

def calculate_dcg(relevances, k):
    """
    Compute DCG@k using gain = 2^rel - 1 and log2 discount.
    """
    relevances = np.asarray(relevances)[:k]
    if len(relevances) == 0:
        return 0.0

    gains = (2 ** relevances) - 1
    discounts = np.log2(np.arange(2, len(relevances) + 2))
    return float(np.sum(gains / discounts))


def calculate_ndcg(relevances, k):
    """
    Compute NDCG@k = DCG@k / IDCG@k
    """
    dcg = calculate_dcg(relevances, k)
    ideal_relevances = sorted(relevances, reverse=True)
    idcg = calculate_dcg(ideal_relevances, k)

    if idcg == 0:
        return 0.0
    return dcg / idcg


In [48]:
# Verify your implementation matches your hand calculation
test_relevances = [1, 2, 1, 0, 2]

dcg = calculate_dcg(test_relevances, k=5)
ndcg = calculate_ndcg(test_relevances, k=5)

print(f"DCG@5 = {dcg:.4f}")
print(f"NDCG@5 = {ndcg:.4f}")

# These should match your hand calculations from Task 4b!

DCG@5 = 4.5533
NDCG@5 = 0.7819


---

## Task 5: Evaluate Your Search Strategy

Now let's evaluate our BM25 search across all queries in the WANDS dataset.

### Evaluation Helper Functions (provided)

In [49]:
# Evaluation helper functions (provided)

def get_relevance_grades(product_ids: list[int], query_id: int, labels_df: pd.DataFrame) -> list[int]:
    """
    Get relevance grades for a list of product IDs given a query.
    
    Args:
        product_ids: List of product IDs in rank order
        query_id: The query ID
        labels_df: DataFrame with relevance labels
        
    Returns:
        List of relevance grades (0, 1, or 2) for each product
    """
    # Get labels for this query
    query_labels = labels_df[labels_df['query_id'] == query_id]
    label_dict = dict(zip(query_labels['product_id'], query_labels['grade']))
    
    # Look up grades for each product (default to 0 if not labeled)
    return [label_dict.get(pid, 0) for pid in product_ids]

def evaluate_single_query(query_text: str, query_id: int, products_df: pd.DataFrame,
                          labels_df: pd.DataFrame, search_func, k: int = 10) -> float:
    """
    Evaluate search for a single query.
    
    Args:
        query_text: The search query text
        query_id: The query ID for looking up labels
        products_df: DataFrame of products
        labels_df: DataFrame with relevance labels
        search_func: Function that takes query and returns DataFrame with product_id column
        k: Number of results to consider
        
    Returns:
        NDCG@k score for this query
    """
    results = search_func(query_text)
    product_ids = results['product_id'].tolist()[:k]
    relevances = get_relevance_grades(product_ids, query_id, labels_df)
    return calculate_ndcg(relevances, k)

def evaluate_search(search_func, products_df: pd.DataFrame, queries_df: pd.DataFrame,
                    labels_df: pd.DataFrame, k: int = 10, verbose: bool = True) -> pd.DataFrame:
    """
    Evaluate search across all queries.
    
    Args:
        search_func: Function that takes query string and returns DataFrame with product_id
        products_df: DataFrame of products
        queries_df: DataFrame of queries
        labels_df: DataFrame with relevance labels
        k: Number of results to consider
        verbose: Whether to print progress
        
    Returns:
        DataFrame with query_id, query, and ndcg columns
    """
    results = []
    
    for _, row in queries_df.iterrows():
        query_id = row['query_id']
        query_text = row['query']
        
        ndcg = evaluate_single_query(query_text, query_id, products_df, 
                                     labels_df, search_func, k)
        results.append({
            'query_id': query_id,
            'query': query_text,
            'ndcg': ndcg
        })
    
    results_df = pd.DataFrame(results)
    
    if verbose:
        print(f"Evaluated {len(results_df)} queries")
        print(f"Mean NDCG@{k}: {results_df['ndcg'].mean():.4f}")
    
    return results_df

print("Evaluation functions defined!")

Evaluation functions defined!


### 5a. Run evaluation on all queries

Create a search function and evaluate it on all queries.

In [51]:
# Load queries
queries = load_wands_queries()
queries.head()


,query_id,query,query_class
0,0,salon chair,Massage Chairs
1,1,smart coffee table,Coffee & Cocktail Tables
2,2,dinosaur,Kids Wall Décor
3,3,turquoise pillows,Accent Pillows
4,4,chair and a half recliner,Recliners


In [53]:
products.head()
queries.head()
labels.head()


,id,query_id,product_id,label,grade
0,0,0,25434,Exact,2
1,1,0,12088,Irrelevant,0
2,2,0,42931,Exact,2
3,3,0,2636,Exact,2
4,4,0,42923,Exact,2


In [52]:
# Task 5a: Run evaluation on all queries

# YOUR CODE HERE
# Task 5a: Compute NDCG@10 for each query

K = 10
ndcg_scores = []

for _, q in queries.iterrows():
    query_id = q["query_id"]
    query_text = q["query"]

    # Run BM25 search
    results = search_products(
        query_text,
        products,
        index,
        doc_lens,
        tokenizer=snowball_tokenize,
        k=K
    )

    # Get relevance labels for this query
    q_labels = labels[labels["query_id"] == query_id]

    if q_labels.empty:
        continue  # no judgments → skip

    # Map product_id → grade
    grade_map = dict(zip(q_labels["product_id"], q_labels["grade"]))

    # Build relevance list in ranking order
    relevances = [
        grade_map.get(pid, 0)  # unjudged → 0
        for pid in results["product_id"]
    ]

    ndcg = calculate_ndcg(relevances, K)
    ndcg_scores.append(ndcg)

# Summary
print(f"Evaluated {len(ndcg_scores)} queries")
print(f"Average NDCG@{K}: {np.mean(ndcg_scores):.4f}")


Evaluated 480 queries
Average NDCG@10: 0.8383


### 5b. Identify failing queries

Find queries where our search performed poorly (NDCG = 0 or very low). Analyze one of them.

In [54]:
# Task 5b: Identify failing queries

# YOUR CODE HERE
K = 10
rows = []

for _, q in queries.iterrows():
    query_id = q["query_id"]
    query_text = q["query"]

    results = search_products(
        query_text,
        products,
        index,
        doc_lens,
        tokenizer=snowball_tokenize,
        k=K
    )

    q_labels = labels[labels["query_id"] == query_id]
    if q_labels.empty:
        continue

    grade_map = dict(zip(q_labels["product_id"], q_labels["grade"]))
    relevances = [grade_map.get(pid, 0) for pid in results["product_id"]]

    ndcg = calculate_ndcg(relevances, K)

    rows.append({
        "query_id": query_id,
        "query": query_text,
        "ndcg@10": ndcg
    })

ndcg_df = pd.DataFrame(rows)

print("Evaluated queries:", len(ndcg_df))
print("Average NDCG@10:", ndcg_df["ndcg@10"].mean().round(4))
ndcg_df.head()


Evaluated queries: 480
Average NDCG@10: 0.8383


,query_id,query,ndcg@10
0,0,salon chair,0.976633
1,1,smart coffee table,0.811487
2,2,dinosaur,1.000000
3,3,turquoise pillows,0.827307
4,4,chair and a half recliner,0.883638


In [55]:
ndcg_df["ndcg@10"].describe()


count    480.000000
mean       0.838250
std        0.244306
min        0.000000
25%        0.785120
50%        0.951375
75%        1.000000
max        1.000000
Name: ndcg@10, dtype: float64

In [56]:
# How many queries are near-perfect vs struggling
bins = [0, 0.2, 0.4, 0.6, 0.8, 0.9, 1.0]
ndcg_df["bucket"] = pd.cut(ndcg_df["ndcg@10"], bins=bins, include_lowest=True)
ndcg_df["bucket"].value_counts().sort_index()


bucket
(-0.001, 0.2]     22
(0.2, 0.4]        13
(0.4, 0.6]        26
(0.6, 0.8]        71
(0.8, 0.9]        70
(0.9, 1.0]       278
Name: count, dtype: int64

In [57]:
print("Top 10 queries by NDCG@10:")
display(ndcg_df.sort_values("ndcg@10", ascending=False).head(10))

print("Bottom 10 queries by NDCG@10:")
display(ndcg_df.sort_values("ndcg@10", ascending=True).head(10))


Top 10 queries by NDCG@10:


,query_id,query,ndcg@10,bucket
2,2,dinosaur,1.0,"(0.9, 1.0]"
463,470,elephant framed,1.0,"(0.9, 1.0]"
8,8,home sweet home sign,1.0,"(0.9, 1.0]"
304,309,moen 5995 arbor one,1.0,"(0.9, 1.0]"
308,313,malachi sled,1.0,"(0.9, 1.0]"
309,314,body pillow and case,1.0,"(0.9, 1.0]"
310,315,medium clips,1.0,"(0.9, 1.0]"
311,316,rolande heavy duty power lift assist recliner,1.0,"(0.9, 1.0]"
312,317,hulmeville writing desk with hutch,1.0,"(0.9, 1.0]"
315,320,blaylock bookcase headboard,1.0,"(0.9, 1.0]"


Bottom 10 queries by NDCG@10:


,query_id,query,ndcg@10,bucket
479,487,rack glass,0.0,"(-0.001, 0.2]"
259,263,small curtain rods,0.0,"(-0.001, 0.2]"
445,452,day bed indian,0.0,"(-0.001, 0.2]"
136,136,black fluffy stool,0.0,"(-0.001, 0.2]"
413,419,living room ideas,0.0,"(-0.001, 0.2]"
125,125,boho bed frame,0.0,"(-0.001, 0.2]"
88,88,rose gold lounge,0.0,"(-0.001, 0.2]"
84,84,full metal bed rose gold,0.0,"(-0.001, 0.2]"
377,383,minnestrista,0.0,"(-0.001, 0.2]"
142,142,kitchen wooden stand,0.0,"(-0.001, 0.2]"


In [58]:
def inspect_query(query_id, k=10):
    qtext = queries.loc[queries["query_id"] == query_id, "query"].iloc[0]
    print("Query:", qtext)

    results = search_products(qtext, products, index, doc_lens, tokenizer=snowball_tokenize, k=k)

    q_labels = labels[labels["query_id"] == query_id][["product_id","label","grade"]]
    grade_map = dict(zip(q_labels["product_id"], q_labels["grade"]))

    out = results[["product_id", "product_name"]].copy()
    out["grade"] = out["product_id"].map(grade_map).fillna(0).astype(int)

    display(out)

# Pick a couple from the bottom
worst_ids = ndcg_df.sort_values("ndcg@10").head(3)["query_id"].tolist()
for qid in worst_ids:
    inspect_query(qid, k=10)


Query: rack glass


,product_id,product_name,grade
19328,19328,hanging wine glass rack,0
7066,7066,opticlean™ 25 compartment glass rack,0
13019,13019,garrow hanging wine glass rack,0
39976,39976,wall mounted wine glass rack,0
32476,32476,36 compartment glass rack,0
32475,32475,25 compartment glass rack,0
12952,12952,signature 8 bottle wall mounted wine bottle ra...,0
32478,32478,16 compartment glass rack,0
7067,7067,opticlean™ 36 compartment glass rack,0
7075,7075,opticlean™ 9 compartment glass rack,0


Query: small curtain rods


,product_id,product_name,grade
29901,29901,wagner shower hooks,0
12315,12315,adjustable tension curtain tension rod,0
34904,34904,premium button shower curtain hooks,0
34907,34907,premium roller shower curtain hooks,0
34903,34903,premium yin shower curtain hooks,0
34902,34902,premium vitalia resin shower curtain hooks,0
34905,34905,premium crystal ball shower curtain hooks,0
34900,34900,premium ora shower curtain hooks,0
34906,34906,premium globe shower curtain hooks,0
34901,34901,premium time 12 piece shower curtain hook set,0


Query: day bed indian


,product_id,product_name,grade
11696,11696,native american healing and protection dreamca...,0
6348,6348,summer 5 graphic art on wrapped canvas,0
6351,6351,summer 5 framed graphic art on wrapped canvas,0
6350,6350,summer 6 graphic art on wrapped canvas,0
6349,6349,summer 6 framed graphic art on wrapped canvas,0
42243,42243,mlb 25-layer stadiumviews lighted end table,0
20622,20622,34.5 '' steel bed frame,0
8062,8062,ellington bed riser,0
5502,5502,square bed risers,0
2050,2050,caulkins heavy duty adjustable metal bed frame...,0


### 5c. Analyze the distribution

Visualize the distribution of NDCG scores.

In [ ]:
# Task 5c: Analyze the distribution

# YOUR CODE HERE

---

## Task 6: Improve Search with Additional Fields

Our baseline only searches the `product_name` field. Let's improve by adding more fields!

### 6a. Index product_class field

The `product_class` field contains the category of the product (e.g., "Rugs", "Coffee Tables"). This is a powerful signal!

Create a search function that combines all three fields (name, description, class).

In [60]:
# Task 6a: Index product_class field

# YOUR CODE HERE
# Task 6a: Index product_class field + build three-field BM25 search

# 1) Build separate corpora (one doc per product per field)
name_docs  = products["product_name"].fillna("").astype(str).tolist()
desc_docs  = products["product_description"].fillna("").astype(str).tolist()
class_docs = products["product_class"].fillna("").astype(str).tolist()

# 2) Build separate inverted indices + doc lengths
name_idx,  name_lens  = build_index(name_docs,  snowball_tokenize)
desc_idx,  desc_lens  = build_index(desc_docs,  snowball_tokenize)
class_idx, class_lens = build_index(class_docs, snowball_tokenize)

print("Built indices:",
      "name terms =", len(name_idx),
      "| desc terms =", len(desc_idx),
      "| class terms =", len(class_idx))

# 3) Three-field search function (BM25 per field, then weighted sum)
def search_products_threefield(
    query: str,
    products_df: pd.DataFrame,
    name_index: dict, name_lengths: list[int],
    desc_index: dict, desc_lengths: list[int],
    class_index: dict, class_lengths: list[int],
    tokenizer,
    k: int = 10,
    w_name: float = 2.0,
    w_desc: float = 1.0,
    w_class: float = 1.5
) -> pd.DataFrame:
    """
    Combine BM25 scores across product_name, product_description, product_class.
    Weighted sum is a simple (and legit) baseline for multi-field retrieval.
    """
    n = len(products_df)

    s_name  = score_bm25(query, name_index,  n, name_lengths,  tokenizer)
    s_desc  = score_bm25(query, desc_index,  n, desc_lengths,  tokenizer)
    s_class = score_bm25(query, class_index, n, class_lengths, tokenizer)

    scores = (w_name * s_name) + (w_desc * s_desc) + (w_class * s_class)

    top_k_idx = np.argsort(-scores)[:k]
    results = products_df.iloc[top_k_idx].copy()
    results["score"] = scores[top_k_idx]
    results["rank"] = range(1, k + 1)
    return results

# Quick smoke test
search_products_threefield(
    "turquoise pillows",
    products,
    name_idx, name_lens,
    desc_idx, desc_lens,
    class_idx, class_lens,
    tokenizer=snowball_tokenize,
    k=10
).head(10)


Built indices: name terms = 25570 | desc terms = 20386 | class terms = 628


,product_id,product_name,product_class,category_hierarchy,product_description,product_features,rating_count,average_rating,review_count,score,rank
5998,5998,noriega turquoise geometric pillow,Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,make a bold statement in your lounge area or b...,covermaterial : polyester/polyester blend|reve...,2.0,4.0,1.0,44.241818,1
19100,19100,turquoise bay sailboat indoor/outdoor throw pi...,Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,approximately 12 inches by 16 inches measured ...,overallheight-toptobottom:12|style : coastal|o...,2.0,5.0,1.0,33.110362,2
27814,27814,turquoise art deco lines pillow cover,Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,makeover and refresh your rooms with just a si...,overallwidth-sidetoside:20|reversesidecolor : ...,4.0,4.5,1.0,31.749875,3
13577,13577,turquoise painted poppy indoor/outdoor throw p...,Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,"wan na transform a serious room into a fun , i...",color : blue|countryoforigin : united states|d...,3.0,5.0,3.0,31.500447,4
23599,23599,patel animal print tortoise turquoise indoor /...,Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,upgrade your outdoor living space with these s...,overalldepth-fronttoback:7|overallproductweigh...,15.0,5.0,11.0,30.667064,5
12436,12436,adaliz fleurs turquoise throw pillow,Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,NaN,fillmaterial : polyester/polyfill|reversesidem...,49.0,4.5,35.0,30.632907,6
21244,21244,"turquoise waters indoor/outdoor , removable co...",Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,transform your outdoor space into an uplifting...,overallwidth-sidetoside:18|overalldepth-frontt...,NaN,NaN,NaN,30.626550,7
11792,11792,buffalo throw pillow,Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,this colorful square reversible buffalo throw ...,contrastingbordercolor : orange|edgetype : pip...,1.0,5.0,1.0,29.310531,8
6874,6874,hames 18 '' throw pillow,Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,this uniquely designed crosshatch decorative p...,pattern : solid color|washingmethod : spot cle...,1.0,5.0,1.0,27.702376,9
19286,19286,oakhill striped lumbar pillow,Accent Pillows,Décor & Pillows / Decorative Pillows & Blanket...,this colorful oblong pillow is bursting with d...,edgetype : fringed edges|subject : no subject|...,NaN,NaN,NaN,26.708981,10


### 6b. Evaluate three-field search

Now evaluate your three-field search on all queries to see how it compares to the baseline.

In [61]:
# Task 6b: Evaluate three-field search

# YOUR CODE HERE
# Task 6b: Evaluate three-field search (and baseline) using NDCG@10

K = 10
rows = []

for _, q in queries.iterrows():
    qid = q["query_id"]
    qtext = q["query"]

    # Baseline (product_name only) — assumes you already built: index, doc_lens
    base_results = search_products(qtext, products, index, doc_lens, tokenizer=snowball_tokenize, k=K)

    # Three-field
    tri_results = search_products_threefield(
        qtext,
        products,
        name_idx, name_lens,
        desc_idx, desc_lens,
        class_idx, class_lens,
        tokenizer=snowball_tokenize,
        k=K
    )

    q_labels = labels[labels["query_id"] == qid]
    if q_labels.empty:
        continue

    grade_map = dict(zip(q_labels["product_id"], q_labels["grade"]))

    base_rels = [grade_map.get(pid, 0) for pid in base_results["product_id"]]
    tri_rels  = [grade_map.get(pid, 0) for pid in tri_results["product_id"]]

    base_ndcg = calculate_ndcg(base_rels, K)
    tri_ndcg  = calculate_ndcg(tri_rels, K)

    rows.append({
        "query_id": qid,
        "query": qtext,
        "baseline_ndcg@10": base_ndcg,
        "threefield_ndcg@10": tri_ndcg,
        "delta": tri_ndcg - base_ndcg
    })

eval_df = pd.DataFrame(rows)

print("Evaluated queries:", len(eval_df))
print("Baseline avg NDCG@10:", eval_df["baseline_ndcg@10"].mean().round(4))
print("Three-field avg NDCG@10:", eval_df["threefield_ndcg@10"].mean().round(4))
print("Avg delta:", eval_df["delta"].mean().round(4))

eval_df.head()


Evaluated queries: 480
Baseline avg NDCG@10: 0.8383
Three-field avg NDCG@10: 0.889
Avg delta: 0.0507


,query_id,query,baseline_ndcg@10,threefield_ndcg@10,delta
0,0,salon chair,0.976633,1.0,0.023367
1,1,smart coffee table,0.811487,1.0,0.188513
2,2,dinosaur,1.000000,1.0,0.000000
3,3,turquoise pillows,0.827307,1.0,0.172693
4,4,chair and a half recliner,0.883638,1.0,0.116362


### 6c. Compare to baseline

Analyze which queries improved and which degraded when using three-field search.

In [62]:
# Task 6c: Compare to baseline

# YOUR CODE HERE
# Task 6c: Compare to baseline (improved vs degraded queries)

improved = (eval_df["delta"] > 0).sum()
degraded = (eval_df["delta"] < 0).sum()
same = (eval_df["delta"] == 0).sum()

print("Improved queries:", improved)
print("Degraded queries:", degraded)
print("Unchanged queries:", same)

print("\nTop 10 improvements:")
display(eval_df.sort_values("delta", ascending=False).head(10)[
    ["query_id", "query", "baseline_ndcg@10", "threefield_ndcg@10", "delta"]
])

print("\nTop 10 degradations:")
display(eval_df.sort_values("delta", ascending=True).head(10)[
    ["query_id", "query", "baseline_ndcg@10", "threefield_ndcg@10", "delta"]
])


Improved queries: 230
Degraded queries: 127
Unchanged queries: 123

Top 10 improvements:


,query_id,query,baseline_ndcg@10,threefield_ndcg@10,delta
445,452,day bed indian,0.000000,1.000000,1.000000
84,84,full metal bed rose gold,0.000000,1.000000,1.000000
142,142,kitchen wooden stand,0.000000,0.984218,0.984218
125,125,boho bed frame,0.000000,0.956009,0.956009
14,14,beds that have leds,0.000000,0.832898,0.832898
259,263,small curtain rods,0.000000,0.745602,0.745602
339,344,ottoman bed queen,0.000000,0.693426,0.693426
442,448,bed side table,0.356207,1.000000,0.643793
136,136,black fluffy stool,0.000000,0.630687,0.630687
388,394,gray leather cocktail table,0.386853,1.000000,0.613147



Top 10 degradations:


,query_id,query,baseline_ndcg@10,threefield_ndcg@10,delta
304,309,moen 5995 arbor one,1.000000,0.000000,-1.000000
394,400,small ladies rocker swivel recliner,0.831555,0.000000,-0.831555
147,147,drum picture,0.978679,0.301030,-0.677649
13,13,outdoor privacy wall,0.867329,0.289065,-0.578264
274,278,parakeet toy,1.000000,0.578674,-0.421326
469,476,hardwood beds,0.775590,0.355023,-0.420567
421,427,lowes tile,0.810409,0.430677,-0.379732
395,401,glass lsmp shades,0.878891,0.509835,-0.369056
436,442,above toilet cabinet,0.998167,0.630930,-0.367237
457,464,cake cutlery,1.000000,0.655944,-0.344056


---

## Task 7: Query Understanding with LLM

Sometimes users search for "star wars rug" when they really want a "rug with Star Wars theme". An LLM can help us understand what the user is actually looking for!

### 7a. Extract product type from query

Write a function using LiteLLM with structured outputs (Pydantic) to extract key information from a query.

In [74]:
# Task 7a: Extract product type, theme, material, color, and any other information you deem relevcant from the query

# YOUR CODE HERE
import json
import re
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any

class QueryUnderstanding(BaseModel):
    product_type: Optional[str] = Field(None, description="Primary product type, e.g., rug, sofa, coffee table, bookshelf")
    theme: Optional[str] = Field(None, description="Theme or style, e.g., star wars, modern, farmhouse")
    material: Optional[str] = Field(None, description="Material, e.g., wood, leather, metal")
    color: Optional[str] = Field(None, description="Color, e.g., blue, turquoise, matte black")
    other: Optional[Dict[str, Any]] = Field(default_factory=dict, description="Any other useful attributes")

def _safe_json_parse(text: str) -> dict:
    """
    Try hard to parse JSON even if the model wraps it in extra text.
    """
    text = text.strip()
    # If it is already JSON
    try:
        return json.loads(text)
    except Exception:
        pass

    # Otherwise try to extract the first JSON object
    m = re.search(r"\{.*\}", text, flags=re.DOTALL)
    if not m:
        raise ValueError("No JSON object found in model output.")
    return json.loads(m.group(0))

def understand_query(query: str, model: str = "gpt-4o-mini") -> QueryUnderstanding:
    """
    Use an LLM to extract structured query info.
    Returns a Pydantic model.
    """
    system = (
        "You extract structured shopping intent from search queries.\n"
        "Return ONLY valid JSON. No commentary.\n"
        "JSON keys: product_type, theme, material, color, other.\n"
        "If unknown, use null."
    )

    user = f"Query: {query}\nReturn JSON now."

    resp = litellm.completion(
        model=model,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        temperature=0.0,
        max_tokens=200,
    )

    raw = resp.choices[0].message.content
    data = _safe_json_parse(raw)
    return QueryUnderstanding(**data)

print("Task 7a: understand_query() defined!")




Task 7a: understand_query() defined!


In [75]:
# Test your query understanding function by running it against these test queries


test_queries = [
    "star wars rug",
    "wooden coffee table",
    "blue leather sofa",
    "modern metal bookshelf"
]

for q in test_queries:
    info = understand_query(q)
    print("\nQuery:", q)
    print(info.model_dump())



Query: star wars rug
{'product_type': 'rug', 'theme': 'star wars', 'material': None, 'color': None, 'other': None}

Query: wooden coffee table
{'product_type': 'coffee table', 'theme': None, 'material': 'wood', 'color': None, 'other': None}

Query: blue leather sofa
{'product_type': 'sofa', 'theme': None, 'material': 'leather', 'color': 'blue', 'other': None}

Query: modern metal bookshelf
{'product_type': 'bookshelf', 'theme': 'modern', 'material': 'metal', 'color': None, 'other': None}


### 7b. Create an LLM-enhanced search

Use the extracted product type to boost matching results. If the LLM identifies "rug" as the product type, boost products where `product_class` contains "rug".

In [78]:
import json, re
from typing import Optional, Dict, Any
from pydantic import BaseModel, Field, field_validator, ValidationError

class QueryUnderstanding(BaseModel):
    product_type: Optional[str] = Field(None)
    theme: Optional[str] = Field(None)
    material: Optional[str] = Field(None)
    color: Optional[str] = Field(None)
    other: Dict[str, Any] = Field(default_factory=dict)

    @field_validator("other", mode="before")
    @classmethod
    def normalize_other(cls, v):
        # Force `other` into a dict no matter what the model returns
        if v is None:
            return {}
        if isinstance(v, dict):
            return v
        if isinstance(v, str):
            return {"notes": v}
        return {"notes": str(v)}

def _safe_json_parse(text: str) -> dict:
    text = text.strip()
    try:
        return json.loads(text)
    except Exception:
        m = re.search(r"\{.*\}", text, flags=re.DOTALL)
        if not m:
            raise ValueError("No JSON object found in model output.")
        return json.loads(m.group(0))

def understand_query(query: str, model: str = "gpt-4o-mini") -> QueryUnderstanding:
    system = (
        "You extract structured shopping intent from search queries.\n"
        "Return ONLY valid JSON. No commentary.\n"
        "Keys: product_type, theme, material, color, other.\n"
        "IMPORTANT: other MUST be an object/dictionary. Put free text into other.notes.\n"
        "If unknown, use null. For other, use {}."
    )
    user = f"Query: {query}\nReturn JSON now."

    resp = litellm.completion(
        model=model,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        temperature=0.0,
        max_tokens=200,
    )

    raw = resp.choices[0].message.content
    data = _safe_json_parse(raw)

    # Hard safety net: never crash the whole run because one query had weird `other`
    try:
        return QueryUnderstanding(**data)
    except ValidationError:
        # salvage what we can and coerce other
        other_val = data.get("other", {})
        if isinstance(other_val, str):
            other_val = {"notes": other_val}
        elif other_val is None:
            other_val = {}
        elif not isinstance(other_val, dict):
            other_val = {"notes": str(other_val)}

        return QueryUnderstanding(
            product_type=data.get("product_type"),
            theme=data.get("theme"),
            material=data.get("material"),
            color=data.get("color"),
            other=other_val
        )

print("Overwrote QueryUnderstanding + understand_query with a robust version.")


Overwrote QueryUnderstanding + understand_query with a robust version.


In [79]:
K = 10
rows = []

for _, qrow in queries.iterrows():
    qid = qrow["query_id"]
    qtext = qrow["query"]

    # Baseline (title-only BM25)
    base = search_products(qtext, products, index, doc_lens, tokenizer=snowball_tokenize, k=K)

    # LLM boosted
    boosted = search_products_llm_boosted(qtext, products, k=K, tokenizer=snowball_tokenize, boost_weight=1.0)

    q_labels = labels[labels["query_id"] == qid]
    if q_labels.empty:
        continue

    grade_map = dict(zip(q_labels["product_id"], q_labels["grade"]))

    base_rels = [grade_map.get(pid, 0) for pid in base["product_id"]]
    boost_rels = [grade_map.get(pid, 0) for pid in boosted["product_id"]]

    rows.append({
        "query_id": qid,
        "query": qtext,
        "baseline_ndcg@10": calculate_ndcg(base_rels, K),
        "llm_boosted_ndcg@10": calculate_ndcg(boost_rels, K),
    })

eval7 = pd.DataFrame(rows)
eval7["delta"] = eval7["llm_boosted_ndcg@10"] - eval7["baseline_ndcg@10"]

print("Evaluated:", len(eval7))
print("Baseline avg NDCG@10:", eval7["baseline_ndcg@10"].mean().round(4))
print("LLM-boosted avg NDCG@10:", eval7["llm_boosted_ndcg@10"].mean().round(4))
print("Avg delta:", eval7["delta"].mean().round(4))

print("\nTop 5 improvements:")
display(eval7.sort_values("delta", ascending=False).head(5))

print("\nTop 5 degradations:")
display(eval7.sort_values("delta", ascending=True).head(5))


ValidationError: 1 validation error for QueryUnderstanding
color
  Input should be a valid string [type=string_type, input_value=['light blue', 'navy blue'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

---
## Task 8: Submit via Pull Request

Now let's submit your work using the Git workflow from previous homeworks.
- [ ] Create a new branch called `homework-3`
- [ ] Commit you work and push it to the branch
- [ ] Create a PR with a nice description of your changes
- [ ] Merge the PR to your main branch
  
**The TA will verify your submission by checking the merged PR on your GitHub repo.**

**Also remember to submit your homework on Blackboard!**
